In [1]:
!unzip /content/datasets.zip

Archive:  /content/datasets.zip
   creating: datasets/images/
   creating: datasets/images/train/
  inflating: datasets/images/train/Cars1.png  
  inflating: datasets/images/train/Cars10.png  
  inflating: datasets/images/train/Cars100.png  
  inflating: datasets/images/train/Cars101.png  
  inflating: datasets/images/train/Cars102.png  
  inflating: datasets/images/train/Cars103.png  
  inflating: datasets/images/train/Cars104.png  
  inflating: datasets/images/train/Cars105.png  
  inflating: datasets/images/train/Cars107.png  
  inflating: datasets/images/train/Cars108.png  
  inflating: datasets/images/train/Cars109.png  
  inflating: datasets/images/train/Cars11.png  
  inflating: datasets/images/train/Cars110.png  
  inflating: datasets/images/train/Cars112.png  
  inflating: datasets/images/train/Cars114.png  
  inflating: datasets/images/train/Cars116.png  
  inflating: datasets/images/train/Cars117.png  
  inflating: datasets/images/train/Cars119.png  
  inflating: datasets/im

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 14.6 MB/s eta 0:00:00


In [1]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.info()

YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
from sklearn.model_selection import train_test_split

images_dir = "images"
annotations_dir = "annotations"
output_dir = "datasets"

os.makedirs(f"{output_dir}/images/train", exist_ok=True)
os.makedirs(f"{output_dir}/images/val", exist_ok=True)
os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
os.makedirs(f"{output_dir}/labels/val", exist_ok=True)

def resize_with_letterbox(image, target_size):
    h, w = image.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    nh, nw = int(h * scale), int(w * scale)
    image_resized = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
    top = (target_size[0] - nh) // 2
    bottom = target_size[0] - nh - top
    left = (target_size[1] - nw) // 2
    right = target_size[1] - nw - left
    image_padded = cv2.copyMakeBorder(image_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    return image_padded, scale, left, top

def convert_annotation(xml_file, scale, left, top, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_labels = []

    for obj in root.findall("object"):
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)

        xmin = int(xmin * scale + left)
        ymin = int(ymin * scale + top)
        xmax = int(xmax * scale + left)
        ymax = int(ymax * scale + top)

        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_labels.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_labels

image_files = [f for f in os.listdir(images_dir) if f.endswith(".png")]
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)


for split, files in [("train", train_files), ("val", val_files)]:
    for image_name in files:

        image_path = os.path.join(images_dir, image_name)
        image = cv2.imread(image_path)


        resized_image, scale, left, top = resize_with_letterbox(image, (640, 640))

        output_image_path = f"{output_dir}/images/{split}/{image_name}"
        cv2.imwrite(output_image_path, resized_image)


        xml_file = os.path.join(annotations_dir, image_name.replace(".png", ".xml"))
        if os.path.exists(xml_file):
            yolo_labels = convert_annotation(xml_file, scale, left, top, 640, 640)


            output_label_path = f"{output_dir}/labels/{split}/{image_name.replace('.png', '.txt')}"
            with open(output_label_path, "w") as f:
                f.write("\n".join(yolo_labels))

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = YOLO("yolov8n.pt").to(device)
model.train(data='placas.yaml', epochs=400, device=device, imgsz=640)
model.export("license_plate_detector")